In [1]:
from selenium import webdriver
import os
from SICOIN_scraper import *
import xlrd
import re
import pandas as pd
import numpy as np

In [ ]:
options = webdriver.ChromeOptions()
options.headless = False # True
# options.add_argument("download.default_directory="+os.getcwd())
options.add_experimental_option("prefs", {
  "download.default_directory": os.path.abspath("../../../Resource Tracking/SICOIN GT/SICOIN - Gestion"),
  "download.prompt_for_download": False,
})
driver = webdriver.Chrome(chrome_options=options)
driver.get('https://sicoin.minfin.gob.gt')

In [8]:
login(driver)
#driver.close()

In [70]:
# IGRC01 - Ejecución Financiera
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh

In [ ]:
# IGRC03 - Comparacion de Ejecucion Financiera y Fisica
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [69]:
# IGRC02 - Ejecución Física
# TB
for year in range(2012, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "17"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [71]:
# IGRC00, IGRC01, IGRC02
# Malaria
for year in range(2013, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "18"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

In [ ]:
# IGRC00, IGRC01, IGRC02
# HIV
for year in range(2013, 2019):
    setYear(driver, year)
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink100"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink101"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    reportTypeA(driver,
        clicks_sequence = ["itemTextLink99", "itemTextLink102"], 
        filters = [{
                "variable": "ENTIDAD",
                "operador": "igual",
                "valor"   : "11130009"
            },
            {
                "variable": "PROGRAMA",
                "operador": "igual",
                "valor"   : "16"
            }
        ])
    # The frames get all messed up, so we simply refresh the ugly webpage
    driver.refresh()

# Preprocessing:

In [131]:
igrc1_options = {
    "categoriesMaxCol": 5,
    "emptyMaxCol": 10,
    "nCategories": 4,
    "dataStartRow": 20,
    "columns": [
            "Program", "Subprograma", "Proyecto", "Actividad-Obra", "RenglonId", "RenglonNombre", 
            "ASIGNADO", "MODIFICADO", "VIGENTE", "DEVENGADO", "PorcentajeEJEC"
        ],
    "expectTotals": True
}

def igrc2_process_row(dataRow):
    dataRow.append("Ejecutado")
    return dataRow
    
def igrc2_process_titleRow(data, categories, dataRow, popCategory):
    if len(dataRow) > 6:
        if len(categories) == 6:
            #print("SWITCH CATS", len(data), categories, dataRow)
            categories[4] = categories[5]
            categories[5] = dataRow[6]
        else:
            categories.append(dataRow[5])
        dataRow2 = categories.copy()
        dataRow2.extend([None, None, dataRow[6], "Inicial"])
        data.append(dataRow2)
        dataRow2 = categories.copy()
        dataRow2.extend([None, None, dataRow[7], "Vigente"])
        data.append(dataRow2)
        dataRow2 = categories.copy()
        dataRow2.extend([None, None, dataRow[8], "Ejecutado"])
        data.append(dataRow2)
        dataRow2 = categories.copy()
        dataRow2.extend([None, None, dataRow[9], "Porcentaje"])
        data.append(dataRow2)
    elif popCategory:
        print(categories)
        if len(categories) == 6:
            categories[3] = categories[5]
            categories.pop()
        categories.pop()
        
igrc2_options = {
    "categoriesMaxCol": 9,
    "emptyMaxCol": 15,
    "nCategories": 6,
    "dataStartRow": 22,
    "columns": [
            "Program", "Subprograma", "Proyecto", "Actividad-Obra", "Concepto", "Unidades", "MuniCode", "Municipality", 
            "Cantidad", "DataType"
        ],
    "spanTitleCols": 2,
    "processTitleRow": igrc2_process_titleRow,
    "processRow": igrc2_process_row
}

igrc3_options = {
    "categoriesMaxCol": 5,
    "emptyMaxCol": 10,
    "nCategories": 4,
    "dataStartRow": 20,
    "columns": [
            "Program", "Subprograma", "Proyecto", "Actividad-Obra", "RenglonId", "RenglonNombre", 
            "ASIGNADO", "MODIFICADO", "VIGENTE", "DEVENGADO", "PorcentajeEJEC"
        ]
}



In [93]:
# a = xlrd.open_workbook("../../../Resource Tracking/SICOIN GT/SICOIN - Gestion/_172.25.3.18reportes$00820850rkxmi5zwc0agaf0veighmspr114316.xls")
a = xlrd.open_workbook("../../../Resource Tracking/SICOIN GT/SICOIN - Gestion/_172.25.3.18reportes$00820967rkxmi5zwc0agaf0veighmspr120557.xls")

In [78]:
s = a.sheet_by_index(0)

In [ ]:
df = SICOIN_extractTable(s, igrc2_options)
df

In [123]:
def nextCell(row, offset = 0):
    lenr = len(row)
    if offset >= lenr:
        return None
    while (offset+1 < lenr) & (row[offset].ctype == xlrd.XL_CELL_EMPTY):
        offset += 1
    return offset

def SICOIN_extractTable(sheet, options):
    print("Extractng ", sheet.nrows, "rows")
    i = options["dataStartRow"]
    data = []
    continuousEmpty = 0
    categories = []
    nColumns = len(options["columns"])
    processRow = "processRow" in options
    oldj = 0
    j0 = 0 
    while True:
        processTitleRow = False
        popCategory = False
        i += 1
        if i >= sheet.nrows:
            break
        row = sheet.row(i)
        j = 0
        j = nextCell(row, j)
        # print(i, categories, ",".join([str(x.value) for x in row]) )
        if j > options["emptyMaxCol"]:
            # empty row
            # print("Empty")
            continuousEmpty += 1
            if continuousEmpty > 20:
                print("Breaking because exceeded cont empty rows limit. Extracted until row", i)
                break;
            continue
        oldj = j0
        j0 = j
        if j < options["categoriesMaxCol"]:
            titleRow = True
            continuousEmpty = 0

            code = str(row[j].value).strip()
            if (options.get("spanTitleCols", 1) > 1):
                for k in range(1,options.get("spanTitleCols")):
                    j = nextCell(row, j+1)
                    code += "-" + str(row[j].value).strip()
            if options.get("expectTotals"): 
                if code.startswith("TOTAL"):
                    if len(categories) > 1:
                        popCategory = True
                        categories.pop()
                    continuousEmpty += 1
                    continue
            else:
                # print("SHOULD POP?", i, j0, oldj, categories)
                if (j0 < oldj):
                    popCategory = True
                    categories.pop()
            if "processTitleRow" in options:
                processTitleRow = True
                # Move on in the row
                j = nextCell(row, j+1)
            categories.append(code)
            if not processTitleRow:
                continue
        continousEmpty = 0
        # print("Data")
        rowData = list.copy(categories)
        while True:
            value = str(row[j].value).strip()
            if len(value) > 0:
                rowData.append(value)
            j = nextCell(row, j+1)
            if j is None:
                break
        
        if processTitleRow:
            options["processTitleRow"](data, categories, rowData, popCategory)
        else:
            if processRow:
                options["processRow"](rowData)
                
            if len(rowData) == nColumns:
                data.append(rowData)
            else:
                print("Bad row: ", i, len(rowData), nColumns, rowData)

    # print(data, nColumns)
    df = pd.DataFrame(data = data, columns = options["columns"])

    return df

In [ ]:
# Read all excel files:
xls_files = []
for root, dirs, files in os.walk("../../../Resource Tracking/SICOIN GT/SICOIN - Gestion"):
    n = len(files)
    for file in files:
        print(file)
        xls_files.append(xlrd.open_workbook(root + "/" + file))

In [ ]:
for x in xls_files:
    s = x.sheet_by_index(0)
    year =  0
    program = None
    dataset = None
    if str(s.row(3)[2].value).endswith("IGRC01"):
        year = s.row(15)[0].value.strip()[-4:]
        dataset = "EjecucionFin"
        program = s.row(13)[2].value
    elif str(s.row(1)[5].value).endswith("IGRC02"):
        year = s.row(15)[8].value
        dataset = "EjecucionFisica"
        program = s.row(8)[5].value
    elif str(s.row(1)[6].value).endswith("IGRC03"):
        year = s.row(15)[10].value
        dataset = "EjecucionFinFis"
        program = s.row(8)[6].value
    else:
        print("Bad report")
    if program:
        program_pre = re.search("entidad (\w* )?\= (\d*)\, programa \= (\d*)", 
                                                program, re.IGNORECASE)
        if program_pre is not None:
            program = "-".join(program_pre.groups()[1:3])
    print(dataset, year, program)

In [156]:
x = xls_files[5]

In [157]:
s = x.sheet_by_index(0)

In [171]:
s.row(13)[2].value, re.search("entidad (\w* )?\= (\d*)\, programa \= (\d*)", s.row(13)[2].value, re.IGNORECASE).groups()

('Entidad Institucional = 11130009, Programa = 17',
 ('Institucional ', '11130009', '17'))